<p><font size="6"><b>Pandas: Combining datasets Part I - concat</b></font></p>

Adapted version from:
> *© 2021, Joris Van den Bossche and Stijn Van Hoey  (<mailto:jorisvandenbossche@gmail.com>, <mailto:stijnvanhoey@gmail.com>). Licensed under [CC BY 4.0 Creative Commons](http://creativecommons.org/licenses/by/4.0/)*

---

In [2]:
import pandas as pd

Combining data is essential functionality in a data analysis workflow. 

Data is distributed in multiple files, different information needs to be merged, new data is calculated, .. and needs to be added together. Pandas provides various facilities for easily combining together Series and DataFrame objects

In [3]:
# redefining the example objects

# series
population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

# dataframe
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

,country,population,area,capital
0,Belgium,11.3,30510,Brussels
1,France,64.3,671308,Paris
2,Germany,81.3,357050,Berlin
3,Netherlands,16.9,41526,Amsterdam
4,United Kingdom,64.9,244820,London


# Adding columns

As we already have seen before, adding a single column is very easy:

In [4]:
pop_density = countries['population']*1e6 / countries['area']

In [5]:
pop_density

0    370.370370
1     95.783158
2    227.699202
3    406.973944
4    265.092721
dtype: float64

In [6]:
countries['pop_density'] = pop_density

In [7]:
countries

,country,population,area,capital,pop_density
0,Belgium,11.3,30510,Brussels,370.370370
1,France,64.3,671308,Paris,95.783158
2,Germany,81.3,357050,Berlin,227.699202
3,Netherlands,16.9,41526,Amsterdam,406.973944
4,United Kingdom,64.9,244820,London,265.092721


Adding multiple columns at once is also possible. For example, the following method gives us a DataFrame of two columns:

In [8]:
countries["country"].str.split(" ", expand=True)

,0,1
0,Belgium,None
1,France,None
2,Germany,None
3,Netherlands,None
4,United,Kingdom


We can add both at once to the dataframe:

In [9]:
countries[['first', 'last']] = countries["country"].str.split(" ", expand=True)

In [10]:
countries

,country,population,area,capital,pop_density,first,last
0,Belgium,11.3,30510,Brussels,370.370370,Belgium,None
1,France,64.3,671308,Paris,95.783158,France,None
2,Germany,81.3,357050,Berlin,227.699202,Germany,None
3,Netherlands,16.9,41526,Amsterdam,406.973944,Netherlands,None
4,United Kingdom,64.9,244820,London,265.092721,United,Kingdom


# Concatenating data

The ``pd.concat`` function does all of the heavy lifting of combining data in different ways.

``pd.concat`` takes a list or dict of Series/DataFrame objects and concatenates them in a certain direction (`axis`) with some configurable handling of “what to do with the other axes”.

## Combining rows - ``pd.concat``

![](../img/pandas/schema-concat0.svg)

Assume we have some similar data as in `countries`, but for a set of different countries:

In [11]:
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

,country,population,area,capital
0,Belgium,11.3,30510,Brussels
1,France,64.3,671308,Paris
2,Germany,81.3,357050,Berlin
3,Netherlands,16.9,41526,Amsterdam
4,United Kingdom,64.9,244820,London


In [12]:
data = {'country': ['Nigeria', 'Rwanda', 'Egypt', 'Morocco', ],
        'population': [182.2, 11.3, 94.3, 34.4],
        'area': [923768, 26338 , 1010408, 710850],
        'capital': ['Abuja', 'Kigali', 'Cairo', 'Rabat']}
countries_africa = pd.DataFrame(data)
countries_africa 

,country,population,area,capital
0,Nigeria,182.2,923768,Abuja
1,Rwanda,11.3,26338,Kigali
2,Egypt,94.3,1010408,Cairo
3,Morocco,34.4,710850,Rabat


We now want to combine the rows of both datasets:

In [13]:
pd.concat([countries, countries_africa])

,country,population,area,capital
0,Belgium,11.3,30510,Brussels
1,France,64.3,671308,Paris
2,Germany,81.3,357050,Berlin
3,Netherlands,16.9,41526,Amsterdam
4,United Kingdom,64.9,244820,London
0,Nigeria,182.2,923768,Abuja
1,Rwanda,11.3,26338,Kigali
2,Egypt,94.3,1010408,Cairo
3,Morocco,34.4,710850,Rabat


If we don't want the index to be preserved:

In [14]:
pd.concat([countries, countries_africa], ignore_index=True)

,country,population,area,capital
0,Belgium,11.3,30510,Brussels
1,France,64.3,671308,Paris
2,Germany,81.3,357050,Berlin
3,Netherlands,16.9,41526,Amsterdam
4,United Kingdom,64.9,244820,London
5,Nigeria,182.2,923768,Abuja
6,Rwanda,11.3,26338,Kigali
7,Egypt,94.3,1010408,Cairo
8,Morocco,34.4,710850,Rabat


When the two dataframes don't have the same set of columns, by default missing values get introduced:

In [15]:
pd.concat([countries, countries_africa[['country', 'capital']]], ignore_index=True)

,country,population,area,capital
0,Belgium,11.3,30510.0,Brussels
1,France,64.3,671308.0,Paris
2,Germany,81.3,357050.0,Berlin
3,Netherlands,16.9,41526.0,Amsterdam
4,United Kingdom,64.9,244820.0,London
5,Nigeria,NaN,NaN,Abuja
6,Rwanda,NaN,NaN,Kigali
7,Egypt,NaN,NaN,Cairo
8,Morocco,NaN,NaN,Rabat


We can also pass a dictionary of objects instead of a list of objects. Now the keys of the dictionary are preserved as an additional index level:

In [16]:
pd.concat({'europe': countries, 'africa': countries_africa})

country  population     area    capital
europe 0         Belgium        11.3    30510   Brussels
       1          France        64.3   671308      Paris
       2         Germany        81.3   357050     Berlin
       3     Netherlands        16.9    41526  Amsterdam
       4  United Kingdom        64.9   244820     London
africa 0         Nigeria       182.2   923768      Abuja
       1          Rwanda        11.3    26338     Kigali
       2           Egypt        94.3  1010408      Cairo
       3         Morocco        34.4   710850      Rabat

<div class="alert alert-info">

**NOTE**:

A typical use case of `concat` is when you create (or read) multiple DataFrame with a similar structure in a loop, and then want to combine this list of DataFrames into a single DataFrame.

For example, assume you have a folder of similar CSV files (eg the data per day) you want to read and combine, this would look like:

```python
import pathlib

data_files = pathlib.Path("data_directory").glob("*.csv")

dfs = []

for path in data_files:
    temp = pd.read_csv(path)
    dfs.append(temp)

df = pd.concat(dfs)
```
<br>
Important: append to a list (not DataFrame), and concat this list at the end after the loop!

</div>

# Joining data with `pd.merge`

Using `pd.concat` above, we combined datasets that had the same columns. But, another typical case is where you want to add information of a second dataframe to a first one based on one of the columns they have in common. That can be done with [`pd.merge`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html).

Let's look again at the titanic passenger data, but taking a small subset of it to make the example easier to grasp:

In [17]:
df = pd.read_csv("data/titanic.csv")
df = df.loc[:9, ['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked']]

In [18]:
df

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0,3,male,22.0,7.2500,S
1,1,1,female,38.0,71.2833,C
2,1,3,female,26.0,7.9250,S
3,1,1,female,35.0,53.1000,S
4,0,3,male,35.0,8.0500,S
5,0,3,male,NaN,8.4583,Q
6,0,1,male,54.0,51.8625,S
7,0,3,male,2.0,21.0750,S
8,1,3,female,27.0,11.1333,S
9,1,2,female,14.0,30.0708,C


Assume we have another dataframe with more information about the 'Embarked' locations:

In [19]:
locations = pd.DataFrame({'Embarked': ['S', 'C', 'N'],
                          'City': ['Southampton', 'Cherbourg', 'New York City'],
                          'Country': ['United Kindom', 'France', 'United States']})

In [20]:
locations

,Embarked,City,Country
0,S,Southampton,United Kindom
1,C,Cherbourg,France
2,N,New York City,United States


We now want to add those columns to the titanic dataframe, for which we can use `pd.merge`, specifying the column on which we want to merge the two datasets:

In [21]:
pd.merge(df, locations, on='Embarked', how='left')

,Survived,Pclass,Sex,Age,Fare,Embarked,City,Country
0,0,3,male,22.0,7.2500,S,Southampton,United Kindom
1,1,1,female,38.0,71.2833,C,Cherbourg,France
2,1,3,female,26.0,7.9250,S,Southampton,United Kindom
3,1,1,female,35.0,53.1000,S,Southampton,United Kindom
4,0,3,male,35.0,8.0500,S,Southampton,United Kindom
5,0,3,male,NaN,8.4583,Q,NaN,NaN
6,0,1,male,54.0,51.8625,S,Southampton,United Kindom
7,0,3,male,2.0,21.0750,S,Southampton,United Kindom
8,1,3,female,27.0,11.1333,S,Southampton,United Kindom
9,1,2,female,14.0,30.0708,C,Cherbourg,France


In this case we use `how='left` (a "left join") because we wanted to keep the original rows of `df` and only add matching values from `locations` to it. Other options are 'inner', 'outer' and 'right' (see the [docs](http://pandas.pydata.org/pandas-docs/stable/merging.html#brief-primer-on-merge-methods-relational-algebra) for more on this, or this visualization: https://joins.spathon.com/).

## Combining columns  - ``pd.concat`` with ``axis=1``

We can use `pd.merge` to combine the columns of two DataFrame based on a common column. If our two DataFrames already have equivalent rows, we can also achieve this basic case using `pd.concat` with specifying `axis=1` (or `axis="columns"`).

![](../img/pandas/schema-concat1.svg)

Assume we have another DataFrame for the same countries, but with some additional statistics:

In [22]:
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

,country,population,area,capital
0,Belgium,11.3,30510,Brussels
1,France,64.3,671308,Paris
2,Germany,81.3,357050,Berlin
3,Netherlands,16.9,41526,Amsterdam
4,United Kingdom,64.9,244820,London


In [23]:
data = {'country': ['Belgium', 'France', 'Netherlands'],
        'GDP': [496477, 2650823, 820726],
        'area': [8.0, 9.9, 5.7]}
country_economics = pd.DataFrame(data).set_index('country')
country_economics

,GDP,area
country,,
Belgium,496477,8.0
France,2650823,9.9
Netherlands,820726,5.7


In [24]:
pd.concat([countries, country_economics], axis=1)

,country,population,area,capital,GDP,area
0,Belgium,11.3,30510.0,Brussels,NaN,NaN
1,France,64.3,671308.0,Paris,NaN,NaN
2,Germany,81.3,357050.0,Berlin,NaN,NaN
3,Netherlands,16.9,41526.0,Amsterdam,NaN,NaN
4,United Kingdom,64.9,244820.0,London,NaN,NaN
Belgium,NaN,NaN,NaN,NaN,496477.0,8.0
France,NaN,NaN,NaN,NaN,2650823.0,9.9
Netherlands,NaN,NaN,NaN,NaN,820726.0,5.7


`pd.concat` matches the different objects based on the index:

In [25]:
countries2 = countries.set_index('country')

In [26]:
countries2

,population,area,capital
country,,,
Belgium,11.3,30510,Brussels
France,64.3,671308,Paris
Germany,81.3,357050,Berlin
Netherlands,16.9,41526,Amsterdam
United Kingdom,64.9,244820,London


In [27]:
pd.concat([countries2, country_economics], axis="columns")

,population,area,capital,GDP,area
country,,,,,
Belgium,11.3,30510,Brussels,496477.0,8.0
France,64.3,671308,Paris,2650823.0,9.9
Germany,81.3,357050,Berlin,NaN,NaN
Netherlands,16.9,41526,Amsterdam,820726.0,5.7
United Kingdom,64.9,244820,London,NaN,NaN
